In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sklearn
sklearn.__version__

'0.20.2'

In [3]:
import pandas as pd
import pickle as pkl

In [4]:
%run utils.ipynb
%run targets.ipynb

In [5]:
file = 'ETHBTC_1m_1519496760000_1549568220000.csv'

In [6]:
# Get Data
df = pd.read_csv('data/'+file)
df.shape
df.head(10)

(500000, 12)

,OPEN_TIME,OPEN,HIGH,LOW,CLOSE,VOLUME,CLOSE_TIME,QUOTE_VOL,NUM_TRADES,TAKE_BASE_VOL,TAKE_QUOTE_VOL,EXTRA
0,1519496760000,0.085541,0.085570,0.085407,0.085473,27.714,1519496819999,2.370345,192,15.308,1.309733,0
1,1519496820000,0.085473,0.085570,0.085448,0.085449,20.128,1519496879999,1.720609,122,6.579,0.562654,0
2,1519496880000,0.085448,0.085569,0.085400,0.085538,45.510,1519496939999,3.890176,209,19.426,1.660960,0
3,1519496940000,0.085538,0.085546,0.085402,0.085540,22.231,1519496999999,1.900513,105,14.320,1.224483,0
4,1519497000000,0.085539,0.085552,0.085464,0.085464,15.243,1519497059999,1.303412,117,8.373,0.716219,0
5,1519497060000,0.085466,0.085555,0.085464,0.085545,31.895,1519497119999,2.727413,131,19.237,1.645547,0
6,1519497120000,0.085545,0.085545,0.085464,0.085539,13.325,1519497179999,1.139346,78,7.311,0.625352,0
7,1519497180000,0.085539,0.085555,0.085414,0.085517,28.202,1519497239999,2.411093,195,14.409,1.232346,0
8,1519497240000,0.085460,0.085545,0.085419,0.085534,15.302,1519497299999,1.307959,134,7.216,0.617035,0
9,1519497300000,0.085440,0.085545,0.085440,0.085533,13.730,1519497359999,1.173580,105,6.186,0.529001,0


In [7]:
data = df.loc[:, ['OPEN_TIME', 'CLOSE_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 6)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464


In [8]:
# Get Target
%run targets.ipynb
target = get_target_1('data/'+file)
target.shape
target.head()

(499999, 2)

,CLOSE_TIME,TARGET
0,1519496819999,1.0
1,1519496879999,0.0
2,1519496939999,0.0
3,1519496999999,1.0
4,1519497059999,0.0


In [9]:
data = pd.merge(data, target, on='CLOSE_TIME')
data.shape
data.head()

(499999, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473,1.0
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449,0.0
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538,0.0
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540,1.0
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464,0.0


In [10]:
TR_split, CV_split, TT_split = TR_CV_TT_split(data, 0.8, 0, 0.2)

TR_split.shape
CV_split.shape
TT_split.shape

TR_split.head()

(399999, 7)

(0, 7)

(100000, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473,1.0
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449,0.0
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538,0.0
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540,1.0
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464,0.0


In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
window = 60
def data_transformer(id, data, target, retrain=False):
    global scaler
    global window
    
    if retrain==True:
        scaler = scaler.fit(data)
        
    data = scaler.transform(data)
    return get_multiple_ts_dataset(id, data, target, tr_win=window, tt_win=0, point_target=True)

In [12]:
TR_id     = TR_split.iloc[:, :2  ].values
TR_data   = TR_split.iloc[:, 2:-1].values
TR_target = TR_split.iloc[:, -1: ].values

TT_id     = TT_split.iloc[:, :2  ].values
TT_data   = TT_split.iloc[:, 2:-1].values
TT_target = TT_split.iloc[:, -1: ].values

TR_id.shape
TR_data.shape
TR_target.shape

TT_id.shape
TT_data.shape
TT_target.shape

(399999, 2)

(399999, 4)

(399999, 1)

(100000, 2)

(100000, 4)

(100000, 1)

In [13]:
%run hyperopt.ipynb
%run models.ipynb

In [14]:
hyperopt = Hyperopt(LR())
results = hyperopt.run(
    TR_id, TR_data, TR_target, 
    eval_set={
        'TT': (TT_id, TT_data, TT_target)
    },
    n_cv=3,
    algo='grid',
    data_transformer=data_transformer,
    max_evals=1
)

Hyperopt Iteration : 1
{'penalty': 'l2', 'C': 3, 'solver': 'sag'}


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 65 seconds


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min finished


('CV_fold : ', 0, 69.84912610054016)
         acc       auc        f1   log_loss
TT  0.515914  0.500754  0.058755  16.719737
TR  0.526623  0.509646  0.186002  16.349890
CV  0.517907  0.503545  0.115081  16.650912


...

('CV_fold : ', 1, 100.6444890499115)
         acc       auc        f1   log_loss
TT  0.515534  0.500499  0.065073  16.732871
TR  0.523676  0.506355  0.144022  16.451681
CV  0.515918  0.498455  0.062116  16.719625


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 110 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.8min finished


('CV_fold : ', 2, 116.5947060585022)
         acc       auc        f1   log_loss
TT  0.515664  0.500565  0.061719  16.728378
TR  0.522619  0.505259  0.132070  16.488171
CV  0.527518  0.501550  0.048535  16.318956
[{'status': 'ok', 'loss': 0.5204790329241281, 'loss_variance': 0.0050514952700074275, 'iteration': 1, 'eval_time': 296.67032980918884, 'params': {'C': 3, 'n_jobs': -1, 'verbose': 1, 'max_iter': 100, 'dual': False, 'class_weight': None, 'warm_start': False, 'intercept_scaling': 1, 'solver': 'sag', 'fit_intercept': True, 'penalty': 'l2', 'multi_class': 'ovr', 'random_state': 0}, 'attributes': [{'n_iter_': array([100], dtype=int32)}, {'n_iter_': array([100], dtype=int32)}, {'n_iter_': array([100], dtype=int32)}]}, {'status': 'ok', 'loss': 0.5205165746893419, 'loss_variance': 0.005075710521933004, 'iteration': 2, 'eval_time': 254.07901191711426, 'params': {'C': 10, 'n_jobs': -1, 'verbose': 1, 'max_iter': 100, 'dual': False, 'class_weight': None, 'warm_start': False, 'intercept_sca

In [27]:
best_params = None
best_loss = 0
for result in results:
    if best_loss < result["loss"]:
        best_loss = result["loss"]
        best_params = result["params"]

best_params
best_loss

{'C': 10,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': -1,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'sag',
 'verbose': 1,
 'warm_start': False}

0.5205165746893419

In [31]:
from sklearn.linear_model import LogisticRegression

TR_id, TR_X, TR_y = data_transformer(TR_id, TR_data, TR_target, retrain=True)
TT_id, TT_X, TT_y = data_transformer(TT_id, TT_data, TT_target, retrain=False)

model = LogisticRegression(**best_params)
model = model.fit(TR_X, TR_y)

TR_y_pred = model.predict_proba(TR_X)[:, 1]
TT_y_pred = model.predict_proba(TT_X)[:, 1]

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(TT_metrics, name='TT'))

print(res)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 112 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.9min finished


         acc       auc        f1   log_loss
TR  0.523268  0.504683  0.116405  16.465746
TT  0.515574  0.500233  0.047793  16.731485


In [ ]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL',
    'win':win
}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'model': 'LR',
    'accuracy':'{:.4f}'.format((CV_metrics['acc']+TT_metrics['acc'])/2)
}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

f = open('models/'+time_tag+'.pkl', 'wb')
pkl.dump(model, f, -1)
f.close()

%notebook -e models/{big_tag}.ipynb